In [27]:
!pip install pandas
!pip install xlrd
!pip install 'pandas-profiling==2.8.0'
!pip install seaborn

You should consider upgrading via the '/Users/omalagonm/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/omalagonm/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/omalagonm/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/omalagonm/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [28]:
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
from matplotlib import pyplot

sns.set_style(style="whitegrid")

In [29]:
df_hurtos = pd.concat([
    pd.read_excel('data/hurto-de-automotores-2017_0.xls', skiprows=9),
    pd.read_excel('data/hurto-de-automotores-2018_0.xls', skiprows=8),
    pd.read_excel('data/hurto-de-automotores-2018_1.xls', skiprows=9),
    pd.read_excel('data/hurto-de-automotores-2019_0.xls', skiprows=8)
])

# Limpieza

In [30]:
# Correción de valores de edad y cantidad. 
# Edad -1 representa que no se conoce la edad del individuo
# Cantidad 1000 se borra en un paso posterior.
df_hurtos['Edad'] = df_hurtos['Edad'].fillna('-1')
df_hurtos['Edad'] = df_hurtos['Edad'].astype(str)
df_hurtos = df_hurtos[df_hurtos['Edad'] != '-']
df_hurtos = df_hurtos[df_hurtos['Edad'] != '-1']

# Corrección de tipos.
cols_str = [
    'Fecha','Hora', 'Departamento', 'Municipio', 'Día', 'Barrio', 'Zona', 'Clase de sitio', 
    'Arma empleada', 'Móvil Agresor', 'Móvil Victima', 'Sexo', 'Estado civil' ,
    'País de nacimiento','Clase de empleado', 'Profesión', 'Escolaridad', 'Código DANE', 'CLASE',
    'MARCA', 'LINEA', 'MODELO', 'COLOR']
for col in cols_str:
    df_hurtos[col] = df_hurtos[col].astype(str)

df_hurtos['Edad'] = df_hurtos['Edad'].astype('int64')

# Remoción de duplicados
df_hurtos.drop_duplicates(inplace=True)

# Remoción de valores atípicos/outliers. 
# 1. Constantes
df_hurtos.drop(columns=['Cantidad', 'Profesión', 'Zona', 'Barrio', 'País de nacimiento'], inplace=True)
# 2. Registros con edad mayor a 110
df_hurtos = df_hurtos[df_hurtos['Edad'] < 110]

# Creación de nuevas columnas
df_hurtos['Fecha'] = df_hurtos['Fecha'].apply(lambda x: x[:11])
df_hurtos['Mes'] = df_hurtos['Fecha'].apply(lambda x: x[5:7])

# Categorización de la variable Hora
def hora_to_cat(hora):
    if hora == 'na':
        return "No_especificada"
    hora = int(hora)

    if hora > 0 and hora <= 4:
        return '(00-04]'
    if hora > 4 and hora <= 8:
        return '(04-08]'
    if hora > 8 and hora <= 12:
        return '(08-12]'
    if hora > 12 and hora <= 16:
        return '(12-16)'
    if hora > 16 and hora <= 20:
        return '(16-20]'
    if hora == 0 or hora > 20:
        return '(20-24]'
df_hurtos['Hora_Cat'] = df_hurtos['Hora'].apply(lambda hora: hora_to_cat(hora[:2]))


In [31]:
colores_dict = {
    'ABANO': 'BLANCO', 'BEIGE': 'BLANCO', 'BLANCO': 'BLANCO', 'BLANCO Y VERDE': 'BLANCO', 'CHAMPAÑA': 'BLANCO', 'CREMA': 'BLANCO', 'MARFIL': 'BLANCO', 'PERLA': 'BLANCO',
    'SAHARA': 'BLANCO', 

    'ACEITUNA': 'VERDE', 'BOSQUE': 'VERDE', 'JADE': 'VERDE', 'LIMON': 'VERDE', 'VERDE': 'VERDE', 'VERDE OSCURO': 'VERDE', 

    'BAMBOO': 'CAFE','CAFE': 'CAFE', 'CARAMELO': 'CAFE', 'HABANO': 'CAFE', 'MARRON': 'CAFE', 'PARDO LAGUNA': 'CAFE', 'ROBLE': 'CAFE', 'TABACO': 'CAFE', 'TERRANOVA': 'CAFE',

    'ACERO': 'GRIS', 'CENIZA GRAFITO': 'GRIS', 'CROMADO': 'GRIS', 'ESTRATO DE PERLA':'GRIS', 'GRAFITO': 'GRIS', 'GRIS': 'GRIS', 'GRIS PLATINO': 'GRIS', 'PLATA': 'GRIS',
    'PLATA NIQUEL': 'GRIS','PLATEADO': 'GRIS', 'PLATINO': 'GRIS', 'STRATO PLATA': 'GRIS', 'SILICE TORNADO': 'GRIS', 'TOLEDANO': 'GRIS',

    'ACUARELA': 'AZUL', 'AZUL': 'AZUL', 'AZUL BAHIA': 'AZUL', 'AZUL BALTICO': 'AZUL', 'AZUL CIELO': 'AZUL', 'AZUL LAGUNA': 'AZUL', 'AZUL LISBOA' : 'AZUL', 
    'AZUL MEDIO': 'AZUL', 'AZUL Y BLANCO': 'AZUL', 'AZUL ZAFIRO': 'AZUL', 'CELESTE': 'AZUL', 'CIELO': 'AZUL', 'TURQUEZA': 'AZUL', 'ZAFIRO': 'AZUL',

    'BRONCE': 'NARANJA', 'COBRE': 'NARANJA', 'FUEGO': 'NARANJA', 'LADRILLO': 'NARANJA', 'MANDARINA': 'NARANJA', 'NARANJA': 'NARANJA', 'OTOÑAL': 'NARANJA', 
    'ZAPOTE': 'NARANJA',

    'BRUMA': 'MORADO', 'FIRMAMENTO': 'MORADO', 'IMPERIAL': 'MORADO', 'LILA': 'MORADO', 'MORADO CORDOBES': 'MORADO', 'VINOTINTO': 'MORADO', 'VIOLETA': 'MORADO', 

    'CEREZA': 'ROJO', 'ROJO': 'ROJO','ROJO - BLANCO': 'ROJO','ROJO-MARRON-ROYAL': 'ROJO','ROJO-VERDE': 'ROJO', 'TERRACOTA': 'ROJO',

    'NEGRO': 'NEGRO', 'NEGRO GRAFITO': 'NEGRO', 'NEGRO Y BLANCO': 'NEGRO', 'NEGRO Y NIQUEL': 'NEGRO',  

    'CINZA EJECUTIVO': 'OTRO', 'CLASICO': 'OTRO', 'COLONIAL': 'OTRO', 'METALIZADO': 'OTRO', 'NO REPORTADO': 'OTRO', 'PRIMER': 'OTRO', 'QUARZO': 'OTRO', 
    'SAFARI': 'OTRO', 'VARIOS COLORES': 'OTRO', '-': 'OTRO',

    'AMARILLO': 'AMARILLO', 'AMARILLO TRANSITO': 'AMARILLO', 'AMARILLO Y BLANCO': 'AMARILLO', 'AMARILLO Y NEGRO': 'AMARILLO', 'DORADO': 'AMARILLO', 
    'MOSTAZA': 'AMARILLO', 'ORO': 'AMARILLO'
}

In [32]:
# Agrupación de colores
df_hurtos['COLOR'] = df_hurtos['COLOR'].apply(lambda c: colores_dict[c])

In [33]:
report = ProfileReport(df_hurtos)
report.to_file('Perfilamiento de hurto a vehículos.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 295.87it/s]
